In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


Using TensorFlow backend.


In [2]:
# load dataset
dataframe = pandas.read_csv('housing.csv',delim_whitespace = True,header = None)
dataframe.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7


In [3]:
# split into input (X) and output(Y) variable
dataset = dataframe.values
X = dataset[:,:13]
Y = dataset[:,13]

In [4]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim = 13, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer = 'normal'))
    # compile model
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn = baseline_model, nb_epoch = 10000, batch_size = 5, verbose = 0)

kfold = KFold(n_splits = 10, random_state = seed)
results = cross_val_score(estimator,X,Y,cv = kfold)
print("Baseline: %.2f (%.2f) MSE" %(results.mean(), results.std()))

Baseline: -114.55 (82.60) MSE


- Running the above code gives us an estimate of the model’s performance on the problem for unseen data. The result reports the mean squared error including the average and standard deviation (average variance) across all 10 folds of the cross validation evaluation.

# Lift Performance By Standardizing The Dataset
- An important concern with the Boston house price dataset is that the input attributes all vary in their scales because they measure di↵erent quantities. It is almost always good practice to prepare your data before modeling it using a neural network model. Continuing on from the above baseline model, we can re-evaluate the same model using a standardized version of the input dataset.
- We can use scikit-learn’s Pipeline framework3 to perform the standardization during the model evaluation process, within each fold of the cross validation. This ensures that there is no data leakage from each testset cross validation fold into the training data. The code below creates a scikit-learn Pipeline that first standardizes the dataset then creates and evaluates the baseline neural network model.

In [5]:
# Regression Example With Boston Dataset: Standardized

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# load dataset
dataframe = pandas.read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]

# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(1, kernel_initializer= 'normal' ))
    # Compile model
    model.compile(loss= 'mean_squared_error' , optimizer= 'adam' )
    return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp',KerasRegressor(build_fn = baseline_model, nb_epoch = 10000, batch_size = 5, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits = 10, random_state = seed)
results = cross_val_score(pipeline,X,Y, cv = kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -567.28 (277.42) MSE


# Evaluate a Deeper Network Topology
- One way to improve the performance of a neural network is to add more layers. This might allow the model to extract and recombine higher order features embedded in the data. In this section we will evaluate the e↵ect of adding one more hidden layer to the model. This is as easy as defining a new function that will create this deeper model, copied from our baseline model above. We can then insert a new line after the first hidden layer. In this case with about half the number of neurons. Our network topology now looks like:

In [6]:
# Regression Example With Boston Dataset: Standardized and Larger
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# load dataset
dataframe = pandas.read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]

# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(6,kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer= 'normal' ))
    # Compile model
    model.compile(loss= 'mean_squared_error' , optimizer= 'adam' )
    return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp',KerasRegressor(build_fn = baseline_model, nb_epoch = 10000, batch_size = 5, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits = 10, random_state = seed)
results = cross_val_score(pipeline,X,Y, cv = kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -572.01 (283.40) MSE


# Evaluate a Wider Network Topology
- Another approach to increasing the representational capacity of the model is to create a wider network. In this section we evaluate the e↵ect of keeping a shallow network architecture and nearly doubling the number of neurons in the one hidden layer. Again, all we need to do is define a new function that creates our neural network model. Here, we have increased the number of neurons in the hidden layer compared to the baseline model from 13 to 20. The topology for our wider network can be summarized as follows:

In [7]:
# Regression Example With Boston Dataset: Standardized and Wider
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# load dataset
dataframe = pandas.read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]

# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=13, kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(1, kernel_initializer= 'normal' ))
    # Compile model
    model.compile(loss= 'mean_squared_error' , optimizer= 'adam' )
    return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp',KerasRegressor(build_fn = baseline_model, nb_epoch = 10000, batch_size = 5, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits = 10, random_state = seed)
results = cross_val_score(pipeline,X,Y, cv = kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -560.82 (273.42) MSE


# Improving Performance by wider,deeper networks and changing number of epochs

In [8]:
# Regression Example With Boston Dataset: Standardized and Wider
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# load dataset
dataframe = pandas.read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]

# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(500, input_dim=13, kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(400,  kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(300,  kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(200,  kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(100,  kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(1, kernel_initializer= 'normal' ))
    # Compile model
    model.compile(loss= 'mean_squared_error' , optimizer= 'adam' )
    return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp',KerasRegressor(build_fn = baseline_model, nb_epoch = 10000, batch_size = 3, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits = 10, random_state = seed)
results = cross_val_score(pipeline,X,Y, cv = kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -29.07 (25.91) MSE


In [10]:
# Regression Example With Boston Dataset: Standardized and Wider
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# load dataset
dataframe = pandas.read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]

# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(500, input_dim=13, kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(400,  kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(300,  kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(200,  kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(100,  kernel_initializer= 'normal' , activation= 'relu' ))
    model.add(Dense(1, kernel_initializer= 'normal' ))
    # Compile model
    model.compile(loss= 'mean_squared_error' , optimizer= 'adam' )
    return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp',KerasRegressor(build_fn = baseline_model, nb_epoch = 100000, batch_size = 3, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits = 10, random_state = seed)
results = cross_val_score(pipeline,X,Y, cv = kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -29.76 (26.39) MSE
